In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import json
import torch

from torch.utils.data.dataloader import DataLoader
from transformers import BertForTokenClassification, AdamW, BertTokenizer, BertTokenizerFast, BatchEncoding, TrainingArguments, Trainer
from dataclasses import dataclass
from typing import List

from ner_training_prep.labelset import LabelSet
from ner_training_prep.containers import TraingingBatch
from ner_training_prep.dataset_ner import TrainingDataset
from ner_training_prep.prepareData import prepare_data

In [4]:
pattern = r' (\b[A-Z][a-z]+\b)(\s\b[A-Z][a-z]+\b)*'
iliad_ner_dataset = prepare_data("../example-texts/iliad.txt", pattern)

In [5]:
## Check out the structure of dataset 
# print(len(iliad_ner_dataset))
# print(iliad_ner_dataset[169])

In [6]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
label_set = LabelSet(labels=["CLEntity"]) #Only one label in this dataset

In [7]:
iliad_ner_trainingData = TrainingDataset(
    data=iliad_ner_dataset, tokenizer=tokenizer, label_set=label_set, tokens_per_batch=16
)

In [8]:
dataset_train = iliad_ner_trainingData[:len(iliad_ner_trainingData) // 2]
dataset_eval = iliad_ner_trainingData[len(iliad_ner_trainingData) // 2:]

In [9]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased", num_labels=len(iliad_ner_trainingData.label_set.ids_to_label.values())
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [10]:
training_args = TrainingArguments("test_trainer")

In [11]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=dataset_train, eval_dataset=dataset_eval
)

In [12]:
trainer.train()

***** Running training *****
  Num examples = 10030
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3762


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
trainer.save_model('bert_ner_finetuned_iliad-with-gpu-pattern2.model')